[View in Colaboratory](https://colab.research.google.com/github/tadasi12/dev/blob/master/yolo_darknet.ipynb)

# ０．準備

## Google Drive をマウントする

In [0]:
# 必要なパッケージを取得
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse

# Colabratory用の認証トークンの生成
# 以下のコードを実行すると認証のリンクが表示され、リンク先に行って認証を行います。
from google.colab import auth
auth.authenticate_user()

Preconfiguring packages ...
Selecting previously unselected package cron.
(Reading database ... 18396 files and directories currently installed.)
Preparing to unpack .../00-cron_3.0pl1-128ubuntu5_amd64.deb ...
Unpacking cron (3.0pl1-128ubuntu5) ...
Selecting previously unselected package libapparmor1:amd64.
Preparing to unpack .../01-libapparmor1_2.11.0-2ubuntu17.1_amd64.deb ...
Unpacking libapparmor1:amd64 (2.11.0-2ubuntu17.1) ...
Selecting previously unselected package libdbus-1-3:amd64.
Preparing to unpack .../02-libdbus-1-3_1.10.22-1ubuntu1_amd64.deb ...
Unpacking libdbus-1-3:amd64 (1.10.22-1ubuntu1) ...
Selecting previously unselected package dbus.
Preparing to unpack .../03-dbus_1.10.22-1ubuntu1_amd64.deb ...
Unpacking dbus (1.10.22-1ubuntu1) ...
Preparing to unpack .../04-gnupg_2.1.15-1ubuntu8.1_amd64.deb ...
Unpacking gnupg (2.1.15-1ubuntu8.1) over (2.1.15-1ubuntu8) ...
Preparing to unpack .../05-gnupg-agent_2.1.15-1ubuntu8.1_amd64.deb ...
Unpacking gnupg-agent (2.1.15-1ubuntu8

In [0]:
# Drive FUSE library用の証明書の生成
# 以下のコードを実行すると認証のリンクが表示され、リンク先で認証を行います。
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

# Google Drive をマウントする
!mkdir -p drive
!google-drive-ocamlfuse drive

## 必要なモジュールをインストールする

### OpenCV

In [29]:
!apt-get update

!apt-get install cmake libeigen3-dev libgtk-3-dev qt5-default freeglut3-dev \
libvtk6-qt-dev libtbb-dev ffmpeg libdc1394-22-dev libavcodec-dev libavformat-dev \
libswscale-dev libjpeg-dev libpng++-dev libtiff5-dev \
libopenexr-dev libwebp-dev libhdf5-dev libpython3.5-dev python3-numpy \
python3-scipy python3-matplotlib libopenblas-dev liblapacke-dev

!add-apt-repository "deb http://security.ubuntu.com/ubuntu xenial-security main"
!apt install libjasper1 libjasper-dev

Hit:1 http://security.ubuntu.com/ubuntu artful-security InRelease
Hit:2 http://security.ubuntu.com/ubuntu xenial-security InRelease
Hit:3 http://archive.ubuntu.com/ubuntu artful InRelease
Hit:4 http://ppa.launchpad.net/alessandro-strada/ppa/ubuntu artful InRelease
Hit:5 http://archive.ubuntu.com/ubuntu artful-updates InRelease
Hit:6 http://archive.ubuntu.com/ubuntu artful-backports InRelease
Reading package lists... Done
Reading package lists... Done
Building dependency tree       
Reading state information... Done
cmake is already the newest version (3.9.1-1).
libjpeg-dev is already the newest version (8c-2ubuntu8).
liblapacke-dev is already the newest version (3.7.1-3ubuntu2).
libopenexr-dev is already the newest version (2.2.0-11ubuntu1).
libwebp-dev is already the newest version (0.6.0-3).
python3-numpy is already the newest version (1:1.12.1-3.1ubuntu4).
ffmpeg is already the newest version (7:3.3.4-2).
freeglut3-dev is already the newest version (2.8.1-3).
libavcodec-dev is alrea

In [34]:
# OpenCV
%%bash
rm -rf opencv
mkdir opencv && cd opencv
wget https://github.com/opencv/opencv/archive/3.3.0.tar.gz
tar xvf 3.3.0.tar.gz

opencv-3.3.0/
opencv-3.3.0/.tgitconfig
opencv-3.3.0/3rdparty/
opencv-3.3.0/3rdparty/carotene/
opencv-3.3.0/3rdparty/carotene/CMakeLists.txt
opencv-3.3.0/3rdparty/carotene/README.md
opencv-3.3.0/3rdparty/carotene/hal/
opencv-3.3.0/3rdparty/carotene/hal/CMakeLists.txt
opencv-3.3.0/3rdparty/carotene/hal/tegra_hal.hpp
opencv-3.3.0/3rdparty/carotene/include/
opencv-3.3.0/3rdparty/carotene/include/carotene/
opencv-3.3.0/3rdparty/carotene/include/carotene/definitions.hpp
opencv-3.3.0/3rdparty/carotene/include/carotene/functions.hpp
opencv-3.3.0/3rdparty/carotene/include/carotene/types.hpp
opencv-3.3.0/3rdparty/carotene/src/
opencv-3.3.0/3rdparty/carotene/src/absdiff.cpp
opencv-3.3.0/3rdparty/carotene/src/accumulate.cpp
opencv-3.3.0/3rdparty/carotene/src/add.cpp
opencv-3.3.0/3rdparty/carotene/src/add_weighted.cpp
opencv-3.3.0/3rdparty/carotene/src/bitwise.cpp
opencv-3.3.0/3rdparty/carotene/src/blur.cpp
opencv-3.3.0/3rdparty/carotene/src/canny.cpp
opencv-3.3.0/3rdparty/carotene/src/channel_extr

--2018-07-15 05:48:33--  https://github.com/opencv/opencv/archive/3.3.0.tar.gz
Resolving github.com (github.com)... 192.30.253.113, 192.30.253.112
Connecting to github.com (github.com)|192.30.253.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://codeload.github.com/opencv/opencv/tar.gz/3.3.0 [following]
--2018-07-15 05:48:33--  https://codeload.github.com/opencv/opencv/tar.gz/3.3.0
Resolving codeload.github.com (codeload.github.com)... 192.30.253.121, 192.30.253.120
Connecting to codeload.github.com (codeload.github.com)|192.30.253.121|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/x-gzip]
Saving to: ‘3.3.0.tar.gz’

     0K .......... .......... .......... .......... .......... 1.92M
    50K .......... .......... .......... .......... .......... 3.78M
   100K .......... .......... .......... .......... .......... 44.6M
   150K .......... .......... .......... .......... .......... 52.0M
   200K

In [44]:
%%bash

rm -rf opencv-3.3.0/build/
mkdir opencv-3.3.0/build && cd opencv-3.3.0/build
cmake -G "Unix Makefiles" --build . \
-D BUILD_CUDA_STUBS=OFF -D BUILD_DOCS=OFF \
-D BUILD_EXAMPLES=OFF -D BUILD_JASPER=OFF -D BUILD_JPEG=OFF -D BUILD_OPENEXR=OFF \
-D BUILD_PACKAGE=ON -D BUILD_PERF_TESTS=OFF -D BUILD_PNG=OFF -D BUILD_SHARED_LIBS=ON \
-D BUILD_TBB=OFF -D BUILD_TESTS=OFF -D BUILD_TIFF=OFF -D BUILD_WITH_DEBUG_INFO=ON \
-D BUILD_ZLIB=OFF -D BUILD_WEBP=OFF -D BUILD_opencv_apps=ON -D BUILD_opencv_calib3d=ON \
-D BUILD_opencv_core=ON -D BUILD_opencv_cudaarithm=OFF -D BUILD_opencv_cudabgsegm=OFF \
-D BUILD_opencv_cudacodec=OFF -D BUILD_opencv_cudafeatures2d=OFF -D BUILD_opencv_cudafilters=OFF \
-D BUILD_opencv_cudaimgproc=OFF -D BUILD_opencv_cudalegacy=OFF -D BUILD_opencv_cudaobjdetect=OFF \
-D BUILD_opencv_cudaoptflow=OFF -D BUILD_opencv_cudastereo=OFF -D BUILD_opencv_cudawarping=OFF \
-D BUILD_opencv_cudev=OFF -D BUILD_opencv_features2d=ON -D BUILD_opencv_flann=ON \
-D BUILD_opencv_highgui=ON -D BUILD_opencv_imgcodecs=ON -D BUILD_opencv_imgproc=ON \
-D BUILD_opencv_java=OFF -D BUILD_opencv_ml=ON -D BUILD_opencv_objdetect=ON \
-D BUILD_opencv_photo=ON -D BUILD_opencv_python2=OFF -D BUILD_opencv_python3=ON \
-D BUILD_opencv_shape=ON -D BUILD_opencv_stitching=ON -D BUILD_opencv_superres=ON \
-D BUILD_opencv_ts=ON -D BUILD_opencv_video=ON -D BUILD_opencv_videoio=ON \
-D BUILD_opencv_videostab=ON -D BUILD_opencv_viz=OFF -D BUILD_opencv_world=OFF \
-D CMAKE_BUILD_TYPE=RELEASE -D WITH_1394=ON -D WITH_CUBLAS=OFF -D WITH_CUDA=OFF \
-D WITH_CUFFT=OFF -D WITH_EIGEN=ON -D WITH_FFMPEG=ON -D WITH_GDAL=OFF -D WITH_GPHOTO2=OFF \
-D WITH_GIGEAPI=ON -D WITH_GSTREAMER=OFF -D WITH_GTK=ON -D WITH_INTELPERC=OFF -D WITH_IPP=ON \
-D WITH_IPP_A=OFF -D WITH_JASPER=ON -D WITH_JPEG=ON -D WITH_LIBV4L=ON -D WITH_OPENCL=ON \
-D WITH_OPENCLAMDBLAS=OFF -D WITH_OPENCLAMDFFT=OFF -D WITH_OPENCL_SVM=OFF -D WITH_OPENEXR=ON \
-D WITH_OPENGL=ON -D WITH_OPENMP=OFF -D WITH_OPENNI=OFF -D WITH_PNG=ON -D WITH_PTHREADS_PF=OFF \
-D WITH_PVAPI=OFF -D WITH_QT=ON -D WITH_TBB=ON -D WITH_TIFF=ON -D WITH_UNICAP=OFF \
-D WITH_V4L=OFF -D WITH_VTK=OFF -D WITH_WEBP=ON -D WITH_XIMEA=OFF -D WITH_XINE=OFF \
-D WITH_LAPACKE=OFF -D WITH_MATLAB=OFF .. 

-- The CXX compiler identification is GNU 7.2.0
-- The C compiler identification is GNU 7.2.0
-- Check for working CXX compiler: /usr/bin/c++
-- Check for working CXX compiler: /usr/bin/c++ -- works
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Check for working C compiler: /usr/bin/cc
-- Check for working C compiler: /usr/bin/cc -- works
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Detecting C compile features
-- Detecting C compile features - done
-- Detected version of GNU GCC: 70 (700)
-- Performing Test HAVE_CXX11 (check file: cmake/checks/cxx11.cpp)
-- Performing Test HAVE_CXX11 - Success
-- Found PythonInterp: /usr/bin/python2.7 (found suitable version "2.7.14", minimum required is "2.7") 
-- Found PythonLibs: /usr/lib/x86_64-linux-gnu/libpython2.7.so (found suitable exact version "2.7.14") 
-- Found PythonInterp: /usr/bin/python3 (found sui

CMake Deprecation Warning at CMakeLists.txt:81 (cmake_policy):
  The OLD behavior for policy CMP0020 will be removed from a future version
  of CMake.

  The cmake-policies(7) manual explains that the OLD behaviors of all
  policies are deprecated and that a policy should be set to OLD only under
  specific short-term circumstances.  Projects should be ported to the NEW
  behavior and not rely on setting a policy to OLD.


CMake Deprecation Warning at CMakeLists.txt:85 (cmake_policy):
  The OLD behavior for policy CMP0022 will be removed from a future version
  of CMake.

  The cmake-policies(7) manual explains that the OLD behaviors of all
  policies are deprecated and that a policy should be set to OLD only under
  specific short-term circumstances.  Projects should be ported to the NEW
  behavior and not rely on setting a policy to OLD.


CMake Deprecation Warning at CMakeLists.txt:94 (cmake_policy):
  The OLD behavior for policy CMP0026 will be removed from a future version
  of CM

In [0]:
%%bash

cd opencv-3.3.0/build
make -j8
make install

### Darknet

In [47]:
%%bash

# DARKNET gcc 5 未満でないと make できない

rm -rf darknet
git clone https://github.com/pjreddie/darknet
#sed -i -e 's/GPU=0/GPU=1/g' darknet/Makefile
#sed -i -e 's/CUDNN=0/CUDNN=1/g' darknet/Makefile
#sed -i -e 's/OPENCV=0/OPENCV=1/g' darknet/Makefile

cd darknet
make

mkdir -p obj
mkdir -p backup
mkdir -p results
gcc -Iinclude/ -Isrc/ -Wall -Wno-unused-result -Wno-unknown-pragmas -Wfatal-errors -fPIC -Ofast -c ./src/gemm.c -o obj/gemm.o
gcc -Iinclude/ -Isrc/ -Wall -Wno-unused-result -Wno-unknown-pragmas -Wfatal-errors -fPIC -Ofast -c ./src/utils.c -o obj/utils.o
gcc -Iinclude/ -Isrc/ -Wall -Wno-unused-result -Wno-unknown-pragmas -Wfatal-errors -fPIC -Ofast -c ./src/cuda.c -o obj/cuda.o
gcc -Iinclude/ -Isrc/ -Wall -Wno-unused-result -Wno-unknown-pragmas -Wfatal-errors -fPIC -Ofast -c ./src/deconvolutional_layer.c -o obj/deconvolutional_layer.o
gcc -Iinclude/ -Isrc/ -Wall -Wno-unused-result -Wno-unknown-pragmas -Wfatal-errors -fPIC -Ofast -c ./src/convolutional_layer.c -o obj/convolutional_layer.o
gcc -Iinclude/ -Isrc/ -Wall -Wno-unused-result -Wno-unknown-pragmas -Wfatal-errors -fPIC -Ofast -c ./src/list.c -o obj/list.o
gcc -Iinclude/ -Isrc/ -Wall -Wno-unused-result -Wno-unknown-pragmas -Wfatal-errors -fPIC -Ofast -c ./src/image.c -o obj/image.o
gcc 

Cloning into 'darknet'...


# １．学習

# ２．認識

In [48]:
%%bash

cd darknet
wget https://pjreddie.com/media/files/yolov3.weights

./darknet detect cfg/yolov3.cfg yolov3.weights data/dog.jpg

data/dog.jpg: Predicted in 10.917943 seconds.
dog: 99%
truck: 92%
bicycle: 99%


--2018-07-15 06:34:50--  https://pjreddie.com/media/files/yolov3.weights
Resolving pjreddie.com (pjreddie.com)... 128.208.3.39
Connecting to pjreddie.com (pjreddie.com)|128.208.3.39|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 248007048 (237M) [application/octet-stream]
Saving to: ‘yolov3.weights’

     0K .......... .......... .......... .......... ..........  0%  737K 5m28s
    50K .......... .......... .......... .......... ..........  0%  739K 5m28s
   100K .......... .......... .......... .......... ..........  0%  103M 3m39s
   150K .......... .......... .......... .......... ..........  0%  126M 2m45s
   200K .......... .......... .......... .......... ..........  0%  742K 3m17s
   250K .......... .......... .......... .......... ..........  0%  129M 2m45s
   300K .......... .......... .......... .......... ..........  0% 90.2M 2m21s
   350K .......... .......... .......... .......... ..........  0%  120M 2m4s
   400K .......... .......... ..........

In [49]:
!ls darknet

backup	  include	 LICENSE.gen   Makefile		results
cfg	  libdarknet.a	 LICENSE.gpl   obj		scripts
darknet   libdarknet.so  LICENSE.meta  predictions.png	src
data	  LICENSE	 LICENSE.mit   python		yolov3.weights
examples  LICENSE.fuck	 LICENSE.v1    README.md


In [2]:
!ls

datalab
